In [1]:
import numpy as np

In [38]:
import cellprofiler
import cellprofiler as cp
import cellprofiler.modules
import cellprofiler.utilities
import cellprofiler.utilities.cpjvm
import glob

In [3]:
import os

In [113]:
# os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-8-openjdk-amd64/jre'
#os.getenv('JAVA_HOME')

In [7]:
import cellprofiler.pipeline

# See headless options at: https://github.com/CellProfiler/CellProfiler/blob/master/cellprofiler/__main__.py
#pipeline = cellprofiler.pipeline.Pipeline()

In [8]:
cp.preferences.set_awt_headless(True)
cp.preferences.set_headless()
cellprofiler.utilities.cpjvm.cp_start_vm()

In [11]:
#pipeline.load('/lab/repos/cytokit/image-quantification-v2.cppipe')

In [12]:
#pipeline.modules()

In [19]:
def get_module_images():
    m = cp.modules.images.Images()
    m.filter_choice.value = cellprofiler.modules.images.FILTER_CHOICE_IMAGES
    return m

def get_module_metadata():
    m = cp.modules.metadata.Metadata()
    return m

def get_module_namesandtypes():
    # Resources:
    # - https://github.com/CellProfiler/CellProfiler/blob/master/tests/modules/test_namesandtypes.py#L960
    m = cp.modules.namesandtypes.NamesAndTypes()
    m.assignment_method.value = cp.modules.namesandtypes.ASSIGN_RULES
    # "Image set matching" -- determines how images from different channels are aligned
    # ORDER means that the file paths must sort alphabetically in a way that will align them
    m.matching_choice.value = cp.modules.namesandtypes.MATCH_BY_ORDER
    m.assignments[0].image_name.value = u'Cell'
    m.assignments[0].rule_filter.value = u'file does contain "cell_mask"'
    m.assignments[0].load_as_choice.value = cp.modules.namesandtypes.LOAD_AS_OBJECTS
    m.add_assignment()
    m.assignments[1].image_name.value = 'Nucleus'
    m.assignments[1].rule_filter.value = 'file does contain "nucleus_mask"'
    m.assignments[1].load_as_choice.value = cp.modules.namesandtypes.LOAD_AS_OBJECTS
    m.add_assignment()
    m.assignments[2].image_name.value = 'CD3'
    m.assignments[2].rule_filter.value = 'file does contain "CD3.tif"'
    m.assignments[2].load_as_choice.value = cp.modules.namesandtypes.LOAD_AS_GRAYSCALE_IMAGE
    m.add_assignment()
    m.assignments[3].image_name.value = 'CD8a'
    m.assignments[3].rule_filter.value = 'file does contain "CD8a.tif"'
    m.assignments[3].load_as_choice.value = cp.modules.namesandtypes.LOAD_AS_GRAYSCALE_IMAGE
    return m

def get_module_measureobjectsizeshape():
    m = cp.modules.measureobjectsizeshape.MeasureObjectSizeShape()
    m.object_groups[0].name.value = 'Cell'
    m.add_object()
    m.object_groups[1].name.value = 'Nucleus'
    return m

def get_pipeline():
    pipeline = cp.pipeline.Pipeline()
    
    m = get_module_images()
    m.module_num = 1
    pipeline.add_module(m)
    
    m = get_module_metadata()
    m.module_num = 2
    pipeline.add_module(m)
    
    m = get_module_namesandtypes()
    m.module_num = 3
    pipeline.add_module(m)
    
    m = get_module_measureobjectsizeshape()
    m.module_num = 4
    pipeline.add_module(m)
    
    return pipeline

In [67]:
pl = get_pipeline()

In [102]:
#p.add_pathnames_to_file_list(['/lab/data/tmp/tile_y00_x00'])

In [68]:
paths = [
    '/lab/data/tmp/tile_y00_x00/object_cell_mask.tif',
    '/lab/data/tmp/tile_y00_x00/object_nucleus_mask.tif',
    '/lab/data/tmp/tile_y00_x00/expression_CD3.tif',
    '/lab/data/tmp/tile_y00_x00/expression_CD8a.tif'
]
paths

['/lab/data/tmp/tile_y00_x00/object_cell_mask.tif',
 '/lab/data/tmp/tile_y00_x00/object_nucleus_mask.tif',
 '/lab/data/tmp/tile_y00_x00/expression_CD3.tif',
 '/lab/data/tmp/tile_y00_x00/expression_CD8a.tif']

In [69]:
pl.add_urls(['file://' + p for p in paths])

In [70]:
pl.file_list

['file:///lab/data/tmp/tile_y00_x00/expression_CD3.tif',
 'file:///lab/data/tmp/tile_y00_x00/expression_CD8a.tif',
 'file:///lab/data/tmp/tile_y00_x00/object_cell_mask.tif',
 'file:///lab/data/tmp/tile_y00_x00/object_nucleus_mask.tif']

In [72]:
m = pl.modules()[0]
m

In [73]:
measurements = cp.measurement.Measurements()
workspace = cp.workspace.Workspace(pl, m, None, None, measurements, None)
file_list = pl.get_filtered_file_list(workspace)

In [74]:
file_list

['file:///lab/data/tmp/tile_y00_x00/expression_CD3.tif',
 'file:///lab/data/tmp/tile_y00_x00/expression_CD8a.tif',
 'file:///lab/data/tmp/tile_y00_x00/object_cell_mask.tif',
 'file:///lab/data/tmp/tile_y00_x00/object_nucleus_mask.tif']

In [75]:
measurements = pl.run()

/lab/repos/CellProfiler/cellprofiler/utilities/hdf5_dict.py:539: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  np.issubdtype(hdf5_type, int) or
/lab/repos/CellProfiler/cellprofiler/utilities/hdf5_dict.py:541: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  hdf5_type_is_float = np.issubdtype(hdf5_type, float)
/opt/conda/envs/cellprofiler/lib/python2.7/site-packages/centrosome/cpmorphology.py:2195: RuntimeWarning: invalid value encountered in true_divide
  ic = np.bincount(labels, i * image) / intensity
/opt/conda/envs/cellprofiler/lib/python2.7/site-packages/centrosome/cpmorphology.py:2196: RuntimeWarning: invalid value encountered in true_divide
  jc = np.bincount(labels, j * image) / intensity
/opt/conda/envs/cellprofiler/

In [78]:
measurements.names

['CD3', 'CD8a']

In [79]:
measurements.object_names

[u'Cell', u'Experiment', u'Image', u'Nucleus']

In [ ]:
m

In [6]:
image_pixel_data = np.array([
    [0, 0, 0, 0, 0, 0, 0],
    [0, 2, 2, 1, 1, 0, 0],
    [0, 2, 2, 1, 1, 0, 0],
    [0, 2, 2, 1, 1, 0, 0],
    [0, 0, 0, 0, 0, 0, 0]], dtype=np.uint8)
image_obj_data = np.array([
    [0, 0, 0, 0, 0, 0, 0],
    [0, 0, 2, 1, 1, 0, 0],
    [0, 2, 2, 1, 1, 0, 0],
    [0, 0, 2, 1, 1, 0, 0],
    [0, 0, 0, 0, 0, 0, 0]], int)

In [7]:
IMAGE_NAME = 'TestImage'
OBJECT_NAME = 'TestObjects'
image = cellprofiler.image.Image(image_pixel_data, convert=False)
#image.pixel_data = image_pixel_data
image_set_list = cellprofiler.image.ImageSetList()
image_set = image_set_list.get_image_set(0)
image_set.add(IMAGE_NAME, image)

object_set = cellprofiler.object.ObjectSet()
objects = cellprofiler.object.Objects()
objects.segmented = image_obj_data
object_set.add_objects(objects, OBJECT_NAME)

module = cellprofiler.modules.measureobjectintensity.MeasureObjectIntensity()
module.images[0].name.value = IMAGE_NAME
module.objects[0].name.value = OBJECT_NAME

measurements = cellprofiler.measurement.Measurements()

ws = cellprofiler.workspace.Workspace(
    cellprofiler.pipeline.Pipeline(),
    module,
    image_set,
    object_set,
    measurements,
    image_set_list
)

In [62]:
ws.file_list.add_files_to_filelist(['file:///tmp/testfile.tif'])

In [64]:
ws.file_list.get_filelist()

[u'file:///tmp/testfile.tif']

In [8]:
image.pixel_data

array([[0, 0, 0, 0, 0, 0, 0],
       [0, 2, 2, 1, 1, 0, 0],
       [0, 2, 2, 1, 1, 0, 0],
       [0, 2, 2, 1, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0]], dtype=uint8)

In [9]:
module.run(ws)

/lab/repos/CellProfiler/cellprofiler/utilities/hdf5_dict.py:539: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  np.issubdtype(hdf5_type, int) or
/lab/repos/CellProfiler/cellprofiler/utilities/hdf5_dict.py:541: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  hdf5_type_is_float = np.issubdtype(hdf5_type, float)


In [10]:
for feature_name in cellprofiler.modules.measureobjectintensity.ALL_MEASUREMENTS:
    feature_name = "%s_%s_%s" % (cellprofiler.modules.measureobjectintensity.INTENSITY, feature_name, IMAGE_NAME)
    data = measurements.get_current_measurement(OBJECT_NAME, feature_name)
    print(feature_name, data)

('Intensity_IntegratedIntensity_TestImage', array([6., 8.]))
('Intensity_MeanIntensity_TestImage', array([1., 2.]))
('Intensity_StdIntensity_TestImage', array([0., 0.]))
('Intensity_MinIntensity_TestImage', array([1., 2.]))
('Intensity_MaxIntensity_TestImage', array([1., 2.]))
('Intensity_IntegratedIntensityEdge_TestImage', array([6., 8.]))
('Intensity_MeanIntensityEdge_TestImage', array([1., 2.]))
('Intensity_StdIntensityEdge_TestImage', array([0., 0.]))
('Intensity_MinIntensityEdge_TestImage', array([1., 2.]))
('Intensity_MaxIntensityEdge_TestImage', array([1., 2.]))
('Intensity_MassDisplacement_TestImage', array([0., 0.]))
('Intensity_LowerQuartileIntensity_TestImage', array([1., 2.]))
('Intensity_MedianIntensity_TestImage', array([1., 2.]))
('Intensity_MADIntensity_TestImage', array([0., 0.]))
('Intensity_UpperQuartileIntensity_TestImage', array([1., 2.]))


In [6]:
pipeline.modules()

In [9]:
from cellprofiler.modules import measureobjectintensity

In [ ]:
measureobjectintensity.MeasureObjectIntensity()

In [7]:
pipeline.savetxt('/tmp/test.cppipe')

In [8]:
!cat /tmp/test.cppipe

CellProfiler Pipeline: http://www.cellprofiler.org
Version:4
DateRevision:318
GitHash:
ModuleCount:8
HasImagePlaneDetails:False

Images:[module_num:1|svn_version:\'Unknown\'|variable_revision_number:2|show_window:False|notes:\x5B\'To begin creating your project, use the Images module to compile a list of files and/or folders that you want to analyze. You can also specify a set of rules to include only the desired files in your selected folders.\'\x5D|batch_state:array(\x5B\x5D, dtype=uint8)|enabled:True|wants_pause:False]
    :\xff\xfe
    Filter images?:\xff\xfeI\x00m\x00a\x00g\x00e\x00s\x00 \x00o\x00n\x00l\x00y\x00
    Select the rule criteria:\xff\xfea\x00n\x00d\x00 \x00(\x00e\x00x\x00t\x00e\x00n\x00s\x00i\x00o\x00n\x00 \x00d\x00o\x00e\x00s\x00 \x00i\x00s\x00i\x00m\x00a\x00g\x00e\x00)\x00 \x00(\x00d\x00i\x00r\x00e\x00c\x00t\x00o\x00r\x00y\x00 \x00d\x00o\x00e\x00s\x00n\x00o\x00t\x00 \x00c\x00o\x00n\x00t\x00a\x00i\x00n\x00r\x00e\x00g\x00e\x00x\x00p\x00 \x00"\x00\x5B\x00\\\x00\\\x00\\\

In [ ]:
pipeline.add_module()